<a href="https://colab.research.google.com/github/Nateme16/geo-aquawatch-water-quality/blob/main/MDNtestsavemodeltest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import tensorflow as tf
print(tf.__version__)
from pprint import pprint
import tensorflow_datasets as tfds

2.8.0


In [ ]:
!git clone https://github.com/gee-community/geo-aquawatch-water-quality/

Cloning into 'geo-aquawatch-water-quality'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 70 (delta 22), reused 38 (delta 9), pack-reused 5
Unpacking objects: 100% (70/70), done.


In [ ]:
def loss(y):
	prior, mu, scale = _parse_outputs(output) 
	distribution = 'MultivariateNormalTriL'

	dist  = getattr(tfp.distributions, distribution)(mu, scale)
	prob  = tfp.distributions.Categorical(probs=prior)
	mix   = tfp.distributions.MixtureSameFamily(prob, dist)

	def impute(mix, y, N):
		return tf.reduce_mean([
			mix.log_prob( tf.where(tf.math.is_nan(y), mix.sample(), y) )
		for _ in range(N)], 0)
	likelihood = mix.log_prob(y)
	return tf.reduce_mean(-likelihood) + tf.add_n([0.])
#		return tf.reduce_mean(-likelihood) + tf.add_n([0.] + self.model.losses)

class MixtureLayer(tf.keras.layers.Layer):

	def __init__(self, n_mix, n_targets, epsilon, **layer_kwargs):
		super(MixtureLayer, self).__init__()
		layer_kwargs.pop('activation', None)

		self.n_mix     = n_mix 
		self.n_targets = n_targets 
		self.epsilon   = tf.constant(epsilon)
		self._layer    = tf.keras.layers.Dense(self.n_outputs, **layer_kwargs)


	@property 
	def layer_sizes(self):
		''' Sizes of the prior, mu, and (lower triangle) scale matrix outputs '''
		sizes = [1, self.n_targets, (self.n_targets * (self.n_targets + 1)) // 2]
		return self.n_mix * np.array(sizes)


	@property 
	def n_outputs(self):
		''' Total output size of the layer object '''
		return sum(self.layer_sizes)


	# @tf.function(experimental_compile=True)
	def call(self, inputs):
		prior, mu, scale = tf.split(self._layer(inputs), self.layer_sizes, axis=1)

		prior = tf.nn.softmax(prior, axis=-1) + tf.constant(1e-9)
		mu    = tf.stack(tf.split(mu, self.n_mix, 1), 1) 
		scale = tf.stack(tf.split(scale, self.n_mix, 1), 1) 
		scale = tfp.math.fill_triangular(scale, upper=False)
		norm  = tf.linalg.diag(tf.ones((1, 1, self.n_targets)))
		sigma = tf.einsum('abij,abjk->abik', tf.transpose(scale, perm=[0,1,3,2]), scale)
		sigma+= self.epsilon * norm
		scale = tf.linalg.cholesky(sigma)

		return tf.keras.layers.concatenate([
			tf.reshape(prior, shape=[-1, self.n_mix]),
			tf.reshape(mu,    shape=[-1, self.n_mix * self.n_targets]),
			tf.reshape(scale, shape=[-1, self.n_mix * self.n_targets ** 2]),
		])

#this is referencing the MDN model Csaba exported to the Github project and connecting the custom objects
model = keras.models.load_model('/content/geo-aquawatch-water-quality/solids/solid_model',custom_objects={ 'loss': loss, 'MixtureLayer': MixtureLayer })

model

In [ ]:
#trying to create a version that I can upload with out custom objects, since, I think we dont need the loss function and the mixture layers should save in model.save in the next code block?
model = keras.models.load_model('/content/geo-aquawatch-water-quality/solids/solid_model',compile=False)



In [ ]:
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Iliw-EMZqMH8Uk2idX9TrNm0xc4043vhp6LvFfjkurE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiM-zWzwtknSOY3iVDL7oIz5bT0bP4sWiOJR0-ZnFsx3JhAu0-1Juc

Successfully saved authorization token.


In [ ]:
#upload to cloud

import folium
print(folium.__version__)

# REPLACE WITH YOUR CLOUD PROJECT!
PROJECT = 'NERecreation'

# Output bucket for trained models.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'eestart_nate'

# This is a good region for hosting AI models.
REGION = 'us-central1'


MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/MDN'
model.save(MODEL_DIR, save_format='tf')



0.8.3


In [ ]:
# Here is where I try to eefy it. Maybe the error is in this input and output dimension settings?

from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "TSS"}) + "'"
print(input_dict)
print(output_dict)

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_pixel_model'

# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}


NameError: ignored

In [ ]:
MODEL_NAME = 'MDN'
VERSION_NAME = 'v0'

PROJECTID= 'ne-recreation-1545342727047'

#!gcloud config set project PROJECT_ID

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECTID} \
  --region {REGION}

#I had to create the model version manually in AI platform. This code wasnt working.

# !gcloud ai-platform versions create {VERSION_NAME} \
#   --project {PROJECTID} \
#   --region {REGION} \
#   --model {MODEL_NAME} \
#   --origin {EEIFIED_DIR} \
#   --framework "TENSORFLOW" \
#   --runtime-version=2.3 \
#   --python-version=3.7

  

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [ne-recreation-1545342727047] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [ ]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_pixel_model'
print(EEIFIED_DIR)

gs://eestart_nate/eeified_pixel_model
